In [1]:
import numpy as np
from scipy.io import loadmat

In [2]:
def predict(train_ind, train_labels, test_ind):
    n = np.empty((test_ind.size, num_classes))

    for i in range(num_classes):
        n[:, i] = alpha[i] + \
            weights[test_ind, :][:, train_ind[(train_labels == i).flatten()]].sum(axis=1)

    d = n.sum(axis=1)
    probs = n / d[:, None]
    
    return probs, n, d

def get_topk(a, k):
    return a[np.argpartition(a, -k)[-k:]]


num_classes = 6
alpha = np.array([0.6, 0.2, 0.05, 0.05, 0.05, 0.05])

data = loadmat('../../data/square_small/square_small_nearest_neighbors.mat')

nns = data['nearest_neighbors'] - 1
sims = data['similarities']
xs = data['x']
labels = data['labels'] - 1

weights = loadmat('../../data/square_small/square_small_weights.mat')['weights'].toarray()

In [5]:
train_ind = np.array([1, 8, 14, 19, 31]) - 1
train_labels = labels[train_ind]
counts = np.zeros(num_classes, dtype=int)
for label in train_labels:
    counts[label] += 1
test_ind = np.delete(np.arange(xs.shape[0]), train_ind)

In [6]:
counts

array([0, 2, 2, 1, 0, 0])

In [7]:
probs, n, d = predict(train_ind, train_labels, test_ind)

In [8]:
sims[np.isin(nns, train_ind)] = 0
sims

array([[0.98817396, 0.        , 0.9842279 ],
       [0.        , 0.9877196 , 0.98028624],
       [0.99296874, 0.98855984, 0.9806326 ],
       [0.98713744, 0.98200077, 0.97965854],
       [0.9926032 , 0.951999  , 0.9505454 ],
       [0.9649241 , 0.96467674, 0.9629477 ],
       [0.99992394, 0.98200077, 0.96811855],
       [0.        , 0.9790579 , 0.975795  ],
       [0.9786599 , 0.97436035, 0.        ],
       [0.9565678 , 0.95580137, 0.95386136],
       [0.9851274 , 0.9629431 , 0.95580137],
       [0.        , 0.98713744, 0.9806326 ],
       [0.99432486, 0.9939973 , 0.98965997],
       [0.9726634 , 0.95720464, 0.9490692 ],
       [0.9970936 , 0.9767694 , 0.9760074 ],
       [0.9857734 , 0.9792407 , 0.96192515],
       [0.99296874, 0.97819126, 0.9737446 ],
       [0.9923714 , 0.98965997, 0.9777405 ],
       [0.9947354 , 0.9941441 , 0.        ],
       [0.9930096 , 0.9851274 , 0.9545671 ],
       [0.99775976, 0.99270767, 0.9737446 ],
       [0.9978266 , 0.9922469 , 0.9861402 ],
       [0.

In [9]:
sims[test_ind, :]

array([[0.        , 0.9877196 , 0.98028624],
       [0.99296874, 0.98855984, 0.9806326 ],
       [0.98713744, 0.98200077, 0.97965854],
       [0.9926032 , 0.951999  , 0.9505454 ],
       [0.9649241 , 0.96467674, 0.9629477 ],
       [0.99992394, 0.98200077, 0.96811855],
       [0.9786599 , 0.97436035, 0.        ],
       [0.9565678 , 0.95580137, 0.95386136],
       [0.9851274 , 0.9629431 , 0.95580137],
       [0.        , 0.98713744, 0.9806326 ],
       [0.99432486, 0.9939973 , 0.98965997],
       [0.9970936 , 0.9767694 , 0.9760074 ],
       [0.9857734 , 0.9792407 , 0.96192515],
       [0.99296874, 0.97819126, 0.9737446 ],
       [0.9923714 , 0.98965997, 0.9777405 ],
       [0.9930096 , 0.9851274 , 0.9545671 ],
       [0.99775976, 0.99270767, 0.9737446 ],
       [0.9978266 , 0.9922469 , 0.9861402 ],
       [0.        , 0.        , 0.98028624],
       [0.9989496 , 0.        , 0.9946926 ],
       [0.99744123, 0.99270767, 0.9859502 ],
       [0.9970936 , 0.9877196 , 0.9705784 ],
       [0.

In [10]:
success_count_bound = sims[test_ind, :][:, :min(sims.shape[1], train_ind.size + 1) + 1].max(axis=1)
success_count_bound

array([0.9877196 , 0.99296874, 0.98713744, 0.9926032 , 0.9649241 ,
       0.99992394, 0.9786599 , 0.9565678 , 0.9851274 , 0.98713744,
       0.99432486, 0.9970936 , 0.9857734 , 0.99296874, 0.9923714 ,
       0.9930096 , 0.99775976, 0.9978266 , 0.98028624, 0.9989496 ,
       0.99744123, 0.9970936 , 0.9775907 , 0.9989496 , 0.9923991 ,
       0.98955727, 0.9922469 , 0.9859502 , 0.9997115 , 0.9997115 ,
       0.9926032 , 0.9930096 , 0.9978266 , 0.99982005, 0.99982005,
       0.99775976, 0.99486756, 0.9923991 , 0.9905908 , 0.99992394,
       0.98855984, 0.99486756, 0.9959961 , 0.9905908 , 0.99159974],
      dtype=float32)

In [11]:
max_alpha = n[:, 1:] + success_count_bound[:, None]
max_alpha

array([[2.18245502, 1.0377196 , 1.0377196 , 1.0377196 , 1.0377196 ],
       [1.19296874, 1.04296874, 1.04296874, 1.04296874, 1.04296874],
       [1.18713744, 1.03713744, 1.03713744, 1.03713744, 1.03713744],
       [1.19260318, 1.04260318, 1.04260318, 1.04260318, 1.04260318],
       [1.1649241 , 1.0149241 , 1.0149241 , 1.0149241 , 1.0149241 ],
       [1.19992394, 1.04992394, 1.04992394, 1.04992394, 1.04992394],
       [1.17865993, 1.02865993, 2.00132333, 1.02865993, 1.02865993],
       [1.15656782, 1.00656782, 1.00656782, 1.00656782, 1.00656782],
       [1.18512739, 1.03512739, 1.03512739, 1.03512739, 1.03512739],
       [2.1753114 , 1.03713744, 1.03713744, 1.03713744, 1.03713744],
       [1.19432486, 1.04432486, 1.04432486, 1.04432486, 1.04432486],
       [1.19709362, 1.04709362, 1.04709362, 1.04709362, 1.04709362],
       [1.18577338, 1.03577338, 1.03577338, 1.03577338, 1.03577338],
       [1.19296874, 1.04296874, 1.04296874, 1.04296874, 1.04296874],
       [1.19237138, 1.04237138, 1.

In [13]:
sigma_primes = []
updated_sigma_primes = []

for class_ in range(1, num_classes):
    sigma_primes.append(get_topk(probs[:, class_], 10).sum())
    
    min_beta = n[:, np.delete(np.arange(num_classes), class_)]
    probabilities = max_alpha[:, class_ - 1] / (max_alpha[:, class_ - 1] + min_beta.sum(axis=1))
    
    updated_sigma_primes.append(get_topk(probabilities, 6).sum() + get_topk(probs[:, class_], 4).sum())
    
sigma_primes = np.array(sigma_primes)
updated_sigma_primes = np.array(updated_sigma_primes)

sigma_primes, updated_sigma_primes

(array([3.32796191, 1.44808278, 0.96841759, 0.5       , 0.5       ]),
 array([6.188692  , 4.61355392, 3.97133795, 3.34974134, 3.34974134]))

In [16]:
bounds = []
bounds.append(np.log(counts[1:] + 1 + sigma_primes).sum())

for class_ in range(1, num_classes):
    bounds.append(
        np.log(counts[class_] + 1 + updated_sigma_primes[class_ - 1])
        + bounds[0] - np.log(counts[class_] + 1 + sigma_primes[class_ - 1])
    )
    
bounds

[5.236410607102059,
 5.609405993891146,
 5.773867509369184,
 5.935352609428408,
 6.301061880064952,
 6.301061880064952]